# Huffman-Codierung

Die Huffman-Codierung ist das optimale verlustfreie Kompressionsverfahren. Es sortiert die Elemente zunächst entsprechend ihrer Auftrittswahrscheinlichkeiten. Dann werden rekursiv die beiden Elemente mit den kleinsten Auftrittswahrscheinlichkeiten zu einem neuen Element zusammengefasst, bis nur noch ein Element übrig bleibt. Es entsteht so ein binärer Baum, an dessen Verzweigungen die Bit 0 bzw. 1 zugeordnet werden. 

In [1]:
# Python-Code zur Huffman-Codierung
# code taken from https://rosettacode.org/wiki/Huffman_coding#Python

from heapq import heappush, heappop, heapify
from collections import defaultdict
import numpy as np

def encode(symb2freq):
    """Huffman encode the given dict mapping symbols to weights"""
    heap = [[wt, [sym, ""]] for sym, wt in symb2freq.items()]
    heapify(heap)
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p))

Im folgenden Abschnitt wird der zu komprimierende Text 

"Auch hier lohnt es sich wieder, auf den Unterschied zwischen den untersuchten Textsorten hinzuweisen, da sich die Hamburger Gruppe nicht allein auf wissenschaftliche Informationstexte beschränkt. Trotzdem sei an dieser Stelle festgehalten, dass informationsdichtere Texte, also Texte mit einer höheren semantischen Redundanz, wie sie der theoretisch-deduktive Ansatz empfiehlt, für gewöhnlich einen behaltensfördernden Effekt erzeugen und sich entsprechend positiv auf die Verständlichkeit eines Textes auswirken – aber auch die Lesezeit unter Umständen wesentlich erhöhen."

definiert und statistisch analysiert.

In [2]:
#zu komprimierender Text 
text = "Auch hier lohnt es sich wieder, auf den Unterschied zwischen den untersuchten Textsorten hinzuweisen, da sich die Hamburger Gruppe nicht allein auf wissenschaftliche Informationstexte beschränkt. Trotzdem sei an dieser Stelle festgehalten, dass informationsdichtere Texte, also Texte mit einer höheren semantischen Redundanz, wie sie der theoretisch-deduktive Ansatz empfiehlt, für gewöhnlich einen behaltensfördernden Effekt erzeugen und sich entsprechend positiv auf die Verständlichkeit eines Textes auswirken – aber auch die Lesezeit unter Umständen wesentlich erhöhen."
text_length_symbols = len(text)

print ("Der Text bestehend aus %g Zeichen soll nun komprimiert werden." % (text_length_symbols))

# Generiere Liste mit Symbolen (Buchstaben) und ihren Auftrittswahrscheinlichkeiten
symbol_count = defaultdict(int)
for ch in text:
    symbol_count[ch] += 1

# Anzahl unterschiedlicher Zeichen in text
symbol_number = len(symbol_count)
print("\nDer Text enthält %g verschiedene Symbole.\n" % (symbol_number))

# Bestimme Auftrittswahrscheinlichkeit der einzelnen Symbole
prob = np.zeros(len(symbol_count))
cntr = 0
for sym, freq in symbol_count.items():
    prob[cntr] = freq / text_length_symbols
    cntr +=1


# Berechne Entropie des Textes
entropy = - np.inner(prob, np.log2(prob))

print("Die Entropie des Alphabetes ist %g bit, die des Textes %g bit.\n" % (entropy, entropy*text_length_symbols) )

Der Text bestehend aus 573 Zeichen soll nun komprimiert werden.

Der Text enthält 42 verschiedene Symbole.

Die Entropie des Alphabetes ist 4.33746 bit, die des Textes 2485.36 bit.



In [8]:
# einfache binäre Codierung ohne Berücksichtigung der Auftrittswahrscheinlichkeiten
equal_symbol_length_bit = np.ceil(np.log2(symbol_number))
equal_text_length_bit = equal_symbol_length_bit * text_length_symbols

print("Bei einer einfachen binären Codierung ohne Berücksichtigung der Auftrittswahrscheinlichkeiten müssten %g bit pro Zeichen aufgewendet werden.\n" % (equal_symbol_length_bit))
print("Der codierte Text hätte eine Länge von %g bit.\n" %(equal_text_length_bit))

Bei einer einfachen binären Codierung ohne Berücksichtigung der Auftrittswahrscheinlichkeiten müssten 6 bit pro Zeichen aufgewendet werden.

Der codierte Text hätte eine Länge von 3438 bit.



Mit dem obigen Python-Code kann nun die Huffman-Codierung durchgeführt werden. Die binären Codeworte sind von links nach rechts zu lesen (im Beispiel der Vorlesung von rechts nach links).

In [3]:
# Rufe Routine für Huffman-Codierung auf
huff = encode(symbol_count)

# Drucke Ergebnis
print("Symbol\tGewicht\tHuffman Code")
code_len = np.zeros(len(symbol_count))
cntr = 0
for p in huff:
    print("%s\t%s\t%s" % (p[0], symbol_count[p[0]], p[1]))
    code_len[cntr] = len(p[1])
    cntr +=1

Symbol	Gewicht	Huffman Code
 	71	011
e	88	110
h	31	0011
i	40	1011
n	45	1111
r	28	0010
s	36	1000
t	38	1001
a	20	10101
c	19	10100
d	23	11101
f	12	00001
l	13	00010
u	16	01010
,	6	000001
m	8	010011
o	10	111001
w	8	010110
x	5	000000
z	7	000111
T	5	0101111
b	4	0100100
g	4	0100101
k	5	1110000
p	5	1110001
ä	3	0001100
ö	4	0101110
.	2	01000000
A	2	01000001
U	2	01000101
v	2	01000111
–	1	00011010
-	1	000110110
E	1	000110111
G	1	010000100
H	1	010000101
I	1	010000110
L	1	010000111
R	1	010001000
S	1	010001001
V	1	010001100
ü	1	010001101


In [6]:
# Berechnung der Entropie des Textes (Annahme: Text ist repräsentativ.)
huff_symbol_length_bit = np.inner(prob,code_len)
huff_text_length_bit = huff_symbol_length_bit*text_length_symbols
print("Die mittlere Wortlänge des Huffman-Codes beträgt %g bit pro Symbol.\n" % (huff_symbol_length_bit))
print("Die Länge des codierten Textes ist %g Bit.\n" % (huff_text_length_bit))

Die mittlere Wortlänge des Huffman-Codes beträgt 4.85515 bit pro Symbol.

Die Länge des codierten Textes ist 2782 Bit.



In [9]:
# Bestimmung der Redundanz des Huffman-Codes
redundancy = huff_symbol_length_bit - entropy
print("Die Redundanz des Huffman-Codes beträgt für dieses Beispiel %g bit oder %g%%.\n" % (redundancy,redundancy/huff_symbol_length_bit*100))
print("Verglichen mit der Codierung ohne Kompression konnte die Datenmenge um den Faktor %g reduziert werden.\n" %(equal_text_length_bit/huff_text_length_bit))

Die Redundanz des Huffman-Codes beträgt für dieses Beispiel 0.517692 bit oder 10.6627%.

Verglichen mit der Codierung ohne Kompression konnte die Datenmenge um den Faktor 1.2358 reduziert werden.



In [10]:
symbol_count

defaultdict(int,
            {'A': 2,
             'u': 16,
             'c': 19,
             'h': 31,
             ' ': 71,
             'i': 40,
             'e': 88,
             'r': 28,
             'l': 13,
             'o': 10,
             'n': 45,
             't': 38,
             's': 36,
             'w': 8,
             'd': 23,
             ',': 6,
             'a': 20,
             'f': 12,
             'U': 2,
             'z': 7,
             'T': 5,
             'x': 5,
             'H': 1,
             'm': 8,
             'b': 4,
             'g': 4,
             'G': 1,
             'p': 5,
             'I': 1,
             'ä': 3,
             'k': 5,
             '.': 2,
             'S': 1,
             'ö': 4,
             'R': 1,
             '-': 1,
             'v': 2,
             'ü': 1,
             'E': 1,
             'V': 1,
             '–': 1,
             'L': 1})